# Install

In [ ]:
!pip3 install pshmodule

In [ ]:
!pip3 install pickle5

In [ ]:
!pip3 install pandas==1.5.0

In [ ]:
!pip3 install swifter

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Load

In [6]:
import sys
sys.path.append('/content/drive/MyDrive/MemeProject/src/pytorch/preprocessing')
print(sys.path)

['/content', '/env/python', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.8/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/MemeProject/src/pytorch/preprocessing']


In [7]:
from tqdm import tqdm
import config as cfg
import pandas as pd
import numpy as np
import swifter
from pshmodule.utils import filemanager as fm

In [8]:
df = fm.load(cfg.origin_class153_ref)

extension : .pickle
Loaded 181704 records from drive/MyDrive/MemeProject/data/augmentation/origin_class153_ref.pickle


In [9]:
df_table = fm.load(cfg.report_label)

extension : .xlsx
Loaded 153 records from drive/MyDrive/MemeProject/data/report/table.xlsx


# Over Sampling

##### table

In [10]:
df_table = df_table[['대화소재/화행', 'count']]

In [11]:
df_table.head()

,대화소재/화행,count
0,"밈봇, 명령",492
1,"밈봇, 혐오",486
2,"밈봇, 실망",480
3,"밈봇, 만족",480
4,"유명인, 칭찬",144


In [12]:
df_table.describe()

,count
count,153.000000
mean,59.803922
std,75.330680
min,6.000000
25%,30.000000
50%,42.000000
75%,72.000000
max,492.000000


##### Utterence Extract

In [13]:
df = df[['num', 'g_num', 'u_num', 'material', 'speech', 'u', 'arg1', 'label']]

In [14]:
df.drop_duplicates(inplace=True)

In [15]:
df.reset_index(drop=True, inplace=True)

In [16]:
df = df[['material', 'speech', 'u', 'arg1', 'label']]

##### Count Check

In [17]:
count_temp = {f"{i[1]['material']}, {i[1]['speech']}, {i[1]['label']}": 0 for i in df.iterrows()}

for i in df.iterrows():
  count_temp[f"{i[1]['material']}, {i[1]['speech']}, {i[1]['label']}"] += 1

In [29]:
sorted_dict = dict(sorted(count_temp.items(), key = lambda item: item[1], reverse = True))
print(sorted_dict)

{'밈봇, 명령, 150': 492, '밈봇, 혐오, 152': 486, '밈봇, 실망, 151': 480, '밈봇, 만족, 149': 480, '유명인, 칭찬, 89': 144, '기후, 진술, 28': 144, '패션, 칭찬, 142': 126, '기념일, 계획, 18': 114, '경사, 자랑, 9': 108, '자기개발, 계획, 107': 102, '전자기기, 칭찬, 118': 102, '자기개발, 의지, 109': 102, '음식, 칭찬, 101': 96, '주거, 소망, 120': 96, '인간관계, 칭찬, 106': 96, '돈, 자랑, 33': 96, '운동, 계획, 78': 96, '문화예술, 칭찬, 41': 90, '연애, 자랑, 75': 90, '취미, 의지, 133': 90, '음료, 의지, 92': 90, '미용, 의지, 50': 90, '직장생활, 의지, 127': 90, '가정생활, 계획, 0': 90, '미디어콘텐츠, 칭찬, 47': 84, '교통, 소망, 13': 78, '여행, 소망, 67': 78, '운동, 의지, 80': 78, '반려동물, 칭찬, 59': 78, '반려동물, 자랑, 57': 78, '연애, 소망, 73': 78, '돈, 소망, 31': 78, '음식, 진술, 100': 72, '여행, 계획, 66': 72, '학교생활, 진술, 147': 72, '미디어콘텐츠, 진술, 46': 72, '학교생활, 계획, 143': 72, '직장생활, 소망, 126': 72, '인간관계, 자랑, 104': 72, '음식, 계획, 96': 66, '돈, 의지, 32': 66, '쇼핑, 진술, 64': 66, '교통, 계획, 12': 66, '학교생활, 자랑, 146': 60, '학교생활, 소망, 144': 60, '가정생활, 자랑, 3': 60, '인간관계, 진술, 105': 60, '음료, 진술, 94': 54, '문화예술, 계획, 36': 54, '미용, 계획, 48': 54, '음식, 소망, 97': 54, '쇼핑, 자랑,

In [41]:
df_count = pd.DataFrame.from_dict(sorted_dict, orient='index')
df_count.reset_index(inplace=True)
df_count.rename(columns={'index':'group', 0:'count'}, inplace=True)

In [42]:
df_count['material'] = df_count.group.swifter.apply(lambda x: x.split(', ')[0])
df_count['speech'] = df_count.group.swifter.apply(lambda x: x.split(', ')[1])
df_count['label'] = df_count.group.swifter.apply(lambda x: x.split(', ')[2])

Pandas Apply:   0%|          | 0/153 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/153 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/153 [00:00<?, ?it/s]

In [44]:
df_count = df_count[['material', 'speech', 'label', 'count']]

In [55]:
df_count.head()

,material,speech,label,count
0,밈봇,명령,150,492
1,밈봇,혐오,152,486
2,밈봇,실망,151,480
3,밈봇,만족,149,480
4,유명인,칭찬,89,144


memebot

In [48]:
df_meme_count = df_count.iloc[:4]

In [50]:
df_meme_count.head()

,material,speech,label,count
0,밈봇,명령,150,492
1,밈봇,혐오,152,486
2,밈봇,실망,151,480
3,밈봇,만족,149,480


remain

In [53]:
df_remain_count = df_count.iloc[4:]

In [56]:
df_remain_count.reset_index(drop=True, inplace=True)

In [57]:
df_remain_count.head()

,material,speech,label,count
0,유명인,칭찬,89,144
1,기후,진술,28,144
2,패션,칭찬,142,126
3,기념일,계획,18,114
4,경사,자랑,9,108
